In [6]:
import pandas as pd

# Adjust pandas options to see all columns and to read subgroup descriptions fully
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 200)

In [15]:
df_s = pd.read_csv("results/subgroup_model_results.csv")
#df_s.columns

In [16]:
# Assign rank: one per row (since you now have only subgroup_model rows)
df_s['subgroup_rank'] = (df_s.index + 1).astype("object")
df_s.loc[df_s['model_type'] == 'global', 'subgroup_rank'] = "N/A"

# Round columns for better readability
for col, ndigits in [('cookD', 2), ('r2', 3), ('mae', 3), ('mse', 3), ('mean_residual', 3),
                     ('global_baseline_r2', 3), ('global_baseline_mae', 3), ('global_baseline_mse', 3), ('global_baseline_mean_residual', 3),
                     ('mean_baseline_r2', 3), ('mean_baseline_mae', 3), ('mean_baseline_mse', 3), ('mean_baseline_mean_residual', 3)]:
    if col in df_s.columns:
        df_s[col] = df_s[col].round(ndigits)

# Compare r2 between subgroup_model and global baseline for each row
df_s['subgroup_r2_better'] = (df_s['r2'] > df_s['global_baseline_r2']).astype("object")
df_s.loc[df_s['model_type'] == 'global', 'subgroup_r2_better'] = None
df_s['subgroup_better_global'] = (df_s['ttest_p'] < 0.1)
df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_global'] = None
df_s['subgroup_better_mean'] = (df_s['ttest_p_mean'] < 0.1)
df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_mean'] = None
df_s['global_better_mean'] = (df_s['ttest_p_mean_global'] < 0.1)
df_s.loc[df_s['model_type'] == 'global', 'global_better_mean'] = None

# Add decision column based on the rules
def determine_decision(row):
    sg = row['subgroup_better_global']
    sm = row['subgroup_better_mean']
    gm = row['global_better_mean']
    
    if sg and sm:
        return "True discovery"
    elif sg and not sm:
        return "True discovery, but model not appropriate"
    elif not sg and sm:
        return "Weak discovery"
    elif not sg and not sm and gm:
        return "False discovery"
    else:
        return "Model not appropriate"

df_s['decision'] = df_s.apply(determine_decision, axis=1)

# Specify desired column order
main_cols = [
    'subgroup_rank', 'model_type', 'description', 'decision', 'subgroup_r2_better', 'cookD', 'n_train', 'n_test',
    'r2', 'global_baseline_r2', 'mean_baseline_r2', 
    'mae', 'global_baseline_mae', 'mean_baseline_mae',
    'mse', 'global_baseline_mse', 'mean_baseline_mse',
    'mean_residual', 'global_baseline_mean_residual', 'mean_baseline_mean_residual'
]
rest_cols = [c for c in df_s.columns if c not in main_cols]
new_order = main_cols + rest_cols

# Reorder columns
df_s = df_s[new_order]

C:\Users\danic\AppData\Local\Temp\ipykernel_21068\2205722591.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_global'] = None
C:\Users\danic\AppData\Local\Temp\ipykernel_21068\2205722591.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_s.loc[df_s['model_type'] == 'global', 'subgroup_better_mean'] = None
C:\Users\danic\AppData\Local\Temp\ipykernel_21068\2205722591.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible

In [17]:
df_s.head(1)

,subgroup_rank,model_type,description,decision,subgroup_r2_better,cookD,n_train,n_test,r2,global_baseline_r2,mean_baseline_r2,mae,global_baseline_mae,mean_baseline_mae,mse,global_baseline_mse,mean_baseline_mse,mean_residual,global_baseline_mean_residual,mean_baseline_mean_residual,intercept,coef__nr_distinct_files_viewed,pval__nr_distinct_files_viewed,coef__total_course_activities,pval__total_course_activities,coef__nightly_activities,pval__nightly_activities,coef__distinct_days,pval__distinct_days,coef__logged_in_weekly,pval__logged_in_weekly,coef__nr_files_viewed,pval__nr_files_viewed,coef__nr_slides_viewed,pval__nr_slides_viewed,ttest_p,ttest_stat,wilcoxon_p,wilcoxon_stat,ttest_p_mean,ttest_stat_mean,wilcoxon_p_mean,wilcoxon_stat_mean,ttest_p_mean_global,ttest_stat_mean_global,wilcoxon_p_mean_global,wilcoxon_stat_mean_global,subgroup_better_global,subgroup_better_mean,global_better_mean
100,N/A,global,NaN,True discovery,None,NaN,1552,388,0.11,NaN,-0.0,1.598,NaN,1.685,3.693,NaN,4.151,0.012,NaN,0.029,4.739552,0.032245,1.285434e-10,0.001919,0.06841,0.00047,0.589183,0.004481,0.615731,-0.246608,0.026818,-0.002626,0.569904,-0.011127,0.000892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Check subgroup models
df_s[['description', 'decision',   
      'r2', 'global_baseline_r2', 'mean_baseline_r2',
      'mse', 'global_baseline_mse', 'mean_baseline_mse',
      'ttest_p', 'n_test', 'subgroup_better_global', 'subgroup_better_mean', 'global_better_mean']].head(100)

1


description  \
0                         course_repeater==np.False_ ∧ croho=='B Electrical Engineering' ∧ origin=='ORIGIN_N'   
1                         course_repeater==np.False_ ∧ croho=='B Electrical Engineering' ∧ origin=='ORIGIN_E'   
2                          croho=='B Electrical Engineering' ∧ origin=='ORIGIN_N' ∧ type_vooropleiding=='VWO'   
3                               croho=='B Computer Science & Engineering' ∧ origin=='ORIGIN_E' ∧ sex=='SEX_M'   
4           course_repeater==np.False_ ∧ croho=='B Electrical Engineering' ∧ type_vooropleiding=='BUITENL_SL'   
5                                                      croho=='B Electrical Engineering' ∧ origin=='ORIGIN_E'   
6                                croho=='B Electrical Engineering' ∧ double_major==False ∧ origin=='ORIGIN_E'   
7                         croho=='B Electrical Engineering' ∧ sex=='SEX_M' ∧ type_vooropleiding=='BUITENL_SL'   
8                   croho=='B Electrical Engineering' ∧ origin=='ORIGIN_E' ∧ type_vooropleiding=='BUITENL_SL'   
9                 croho=='B Computer Science & Engineering' ∧ sex=='SEX_M' ∧ type_vooropleiding=='BUITENL_SL'   
10                                       croho=='B Electrical Engineering' ∧ type_vooropleiding=='BUITENL_SL'   
11                 croho=='B Electrical Engineering' ∧ double_major==False ∧ type_vooropleiding=='BUITENL_SL'   
12                course_repeater==np.False_ ∧ croho=='B Computer Science & Engineering' ∧ origin=='ORIGIN_E'   
13                                              croho=='B Electrical Engineering' ∧ type_vooropleiding=='VWO'   
14                        croho=='B Electrical Engineering' ∧ double_major==False ∧ type_vooropleiding=='VWO'   
15          croho=='B Computer Science & Engineering' ∧ origin=='ORIGIN_E' ∧ type_vooropleiding=='BUITENL_SL'   
16                                             croho=='B Computer Science & Engineering' ∧ origin=='ORIGIN_E'   
17                       croho=='B Computer Science & Engineering' ∧ double_major==False ∧ origin=='ORIGIN_E'   
18                                                     croho=='B Electrical Engineering' ∧ origin=='ORIGIN_N'   
19                               croho=='B Electrical Engineering' ∧ double_major==False ∧ origin=='ORIGIN_N'   
20                                                                                  type_vooropleiding=='HBO'   
21                                                            double_major==False ∧ type_vooropleiding=='HBO'   
22                                      croho=='B Electrical Engineering' ∧ origin=='ORIGIN_E' ∧ sex=='SEX_M'   
23                                                                         croho=='B Psychology & Technology'   
24                                                   croho=='B Psychology & Technology' ∧ double_major==False   
25  course_repeater==np.False_ ∧ croho=='B Computer Science & Engineering' ∧ type_vooropleiding=='BUITENL_SL'   
26                                      course_repeater==np.False_ ∧ sex=='SEX_V' ∧ type_vooropleiding=='VWO'   
27                  croho=='B Electrical Engineering' ∧ origin=='ORIGIN_R' ∧ type_vooropleiding=='BUITENL_SL'   
28                      course_repeater==np.False_ ∧ croho=='B Computer Science & Engineering' ∧ sex=='SEX_M'   
29                               croho=='B Industrial Engineering' ∧ sex=='SEX_M' ∧ type_vooropleiding=='VWO'   
30                 course_repeater==np.False_ ∧ croho=='B Industrial Engineering' ∧ type_vooropleiding=='VWO'   
31                               croho=='B Computer Science & Engineering' ∧ type_vooropleiding=='BUITENL_SL'   
32         croho=='B Computer Science & Engineering' ∧ double_major==False ∧ type_vooropleiding=='BUITENL_SL'   
33                                              origin=='ORIGIN_N' ∧ sex=='SEX_V' ∧ type_vooropleiding=='VWO'   
34                                      croho=='B Electrical Engineering' ∧ origin=='ORIGIN_N' ∧ sex=='SEX_M'   
35                            

In [5]:
# Compare individual subgroup against global model

subgroup_rank = 1

df_individual = df_s[(df_s['model_type'] == 'global') | (df_s['subgroup_rank'] == subgroup_rank)]

cols = ['subgroup_rank', 'model_type', 'description', 'cookD', 'n_train', 'n_test'] + rest_cols

df_individual[cols]

,subgroup_rank,model_type,description,cookD,n_train,n_test,intercept,coef__G1,pval__G1,coef__G2,pval__G2,coef__absences,pval__absences,coef__activities,pval__activities,coef__paid,pval__paid,coef__famsup,pval__famsup,coef__schoolsup,pval__schoolsup,coef__studytime,pval__studytime,indices,ttest_p,ttest_stat,wilcoxon_p,wilcoxon_stat,ttest_p_mean,ttest_stat_mean,wilcoxon_p_mean,wilcoxon_stat_mean,ttest_p_mean_global,ttest_stat_mean_global,wilcoxon_p_mean_global,wilcoxon_stat_mean_global,subgroup_better_global,subgroup_better_mean,global_better_mean
0,1,subgroup,"Walc in (2.0, 3.0] ∧ nursery=='yes' ∧ school=='MS'",474.33,29,5,-2.790428,0.795457,0.000344,0.393798,3.085668e-02,-0.049514,0.693350,0.714396,0.292423,-5.530959,0.002976,1.967350,0.030990,-9.398559,0.000059,-0.003401,0.994783,"[4, 8, 18, 36, 46, 53, 58, 75, 83, 95, 97, 118, 155, 176, 215, 221, 227, 235, 249, 290, 313, 363, 365, 440, 447, 457, 461, 486, 504]",0.865752,1.283915,0.90625,12.0,0.106733,-1.478048,0.21875,4.0,0.089036,-1.631734,0.0625,1.0,False,False,True
50,N/A,global,NaN,NaN,519,130,-0.330637,0.158945,0.000077,0.887581,8.869635e-84,0.017393,0.153938,-0.079296,0.471657,-0.343987,0.124951,0.122758,0.278713,-0.114223,0.539993,0.041240,0.564622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
subgroup_linear_models = pd.read_csv("results/subgroup_portuguese_linear.csv")

In [7]:
subgroup_linear_models[['subgroup', 'n', 'cookD']].drop_duplicates().iloc[[14, 18, 25, 28, 29, 38, 44, 45, 46]]#[8,9, 27, 28, 41, 47]]

,subgroup,n,cookD
126,"Mjob=='other' ∧ Pstatus=='T' ∧ Walc in (3.0, 5.0]",25,240.534050
162,"Walc in (2.0, 3.0] ∧ sex=='M' ∧ traveltime in (0.999, 2.0]",25,234.751851
225,"Dalc in (2.0, 5.0] ∧ famrel in (0.999, 4.0] ∧ school=='GP'",24,218.148582
252,"goout in (4.0, 5.0] ∧ higher=='yes' ∧ reason=='course'",29,215.428720
261,"Walc in (3.0, 5.0] ∧ guardian=='father'",23,211.476044
342,"Mjob=='other' ∧ Walc in (3.0, 5.0] ∧ higher=='yes'",23,195.506601
396,"Pstatus=='T' ∧ freetime in (4.0, 5.0] ∧ reason=='course'",20,190.188238
405,"Walc in (2.0, 3.0] ∧ higher=='yes' ∧ romantic=='no'",40,189.731408
414,"Medu in (3.0, 4.0] ∧ age in (17.0, 18.0] ∧ nursery=='yes'",23,189.182909


In [8]:
subgroup_linear_models

,subgroup,n,cookD,indices,term,coef_group,se_group,t_group,p_group,sig_group,coef_global,se_global,t_global,p_global,sig_global
0,"Dalc in (2.0, 5.0] ∧ reason=='course'",20,709.810622,"[25, 31, 38, 55, 74, 75, 123, 137, 154, 170, 182, 231, 291, 302, 342, 348, 366, 419, 429, 444]",Intercept,-3.448716,3.816816,-0.903558,0.385590,,0.027177,0.268016,0.101401,9.192777e-01,
1,"Dalc in (2.0, 5.0] ∧ reason=='course'",20,709.810622,"[25, 31, 38, 55, 74, 75, 123, 137, 154, 170, 182, 231, 291, 302, 342, 348, 366, 419, 429, 444]",G1,1.491312,0.892413,1.671100,0.122876,,0.132953,0.039030,3.406398,7.180220e-04,***
2,"Dalc in (2.0, 5.0] ∧ reason=='course'",20,709.810622,"[25, 31, 38, 55, 74, 75, 123, 137, 154, 170, 182, 231, 291, 302, 342, 348, 366, 419, 429, 444]",G2,0.002628,0.766467,0.003429,0.997325,,0.890912,0.036540,24.381771,5.217999e-84,***
3,"Dalc in (2.0, 5.0] ∧ reason=='course'",20,709.810622,"[25, 31, 38, 55, 74, 75, 123, 137, 154, 170, 182, 231, 291, 302, 342, 348, 366, 419, 429, 444]",absences,0.419381,0.174835,2.398725,0.035312,*,0.010979,0.012097,0.907587,3.645878e-01,
4,"Dalc in (2.0, 5.0] ∧ reason=='course'",20,709.810622,"[25, 31, 38, 55, 74, 75, 123, 137, 154, 170, 182, 231, 291, 302, 342, 348, 366, 419, 429, 444]",activities,-0.794305,1.403415,-0.565980,0.582769,,-0.051632,0.109035,-0.473532,6.360657e-01,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,"Pstatus=='T' ∧ Walc in (3.0, 5.0] ∧ nursery=='no'",23,187.598493,"[1, 20, 33, 35, 66, 68, 74, 83, 147, 154, 216, 321, 331, 334, 342, 348, 383, 399, 402, 419, 427, 429, 431]",activities,0.200127,0.570490,0.350798,0.730962,,-0.051632,0.109035,-0.473532,6.360657e-01,
446,"Pstatus=='T' ∧ Walc in (3.0, 5.0] ∧ nursery=='no'",23,187.598493,"[1, 20, 33, 35, 66, 68, 74, 83, 147, 154, 216, 321, 331, 334, 342, 348, 383, 399, 402, 419, 427, 429, 431]",paid,0.326356,1.092965,0.298597,0.769634,,-0.449179,0.221763,-2.025495,4.341291e-02,*
447,"Pstatus=='T' ∧ Walc in (3.0, 5.0] ∧ nursery=='no'",23,187.598493,"[1, 20, 33, 35, 66, 68, 74, 83, 147, 154, 216, 321, 331, 334, 342, 348, 383, 399, 402, 419, 427, 429, 431]",famsup,1.108130,0.617034,1.795898,0.094117,.,0.098840,0.113121,0.873762,3.827193e-01,
448,"Pstatus=='T' ∧ Walc in (3.0, 5.0] ∧ nursery=='no'",23,187.598493,"[1, 20, 33, 35, 66, 68, 74, 83, 147, 154, 216, 321, 331, 334, 342, 348, 383, 399, 402, 419, 427, 429, 431]",schoolsup,5.083626,1.357058,3.746063,0.002170,**,0.027472,0.184330,0.149034,8.815941e-01,
